In [115]:
import numpy as np
from collections import Counter
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

In [118]:
def regression_metrics(y_test, y_pred):
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
    print(f'MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'RMSE: {(mean_squared_error(y_test, y_pred))**0.5}')
    print(f'MAPE: {(mean_absolute_percentage_error(y_test, y_pred))**0.5}')
    print(f'R^2: {r2_score(y_test, y_pred)}')

In [101]:
class CART:

    def __init__(self,
                 max_depth: int | None = None, 
                 min_samples_split: int = 2,
                 classification: bool = False,
                 ) -> None:
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.classification = classification

    class Node:
        def __init__(self, 
                     feature: int | None = None, 
                     threshold: float | None = None, 
                     value: int | float | None = None, 
                     left = None, right = None
                     ) -> None:
                self.feature = feature
                self.threshold = threshold
                self.value = value
                self.left = left
                self.right = right

    def _entropy(self, Y: np.ndarray) -> float:
        """Находит энтропию столбца"""
        probabilities = np.array(list(Counter(Y).values())) / len(Y)
        return -np.sum(probabilities * np.log2(probabilities))
    
    def _MSE(self, Y: np.ndarray) -> float:
        """Находит среднеквадратичную ошибку столбца"""
        return np.mean((Y - np.mean(Y))**2)

    def _split_dataset(self, X: np.ndarray, Y: np.ndarray, feature: int, threshold: float):
        """
        Разделяет датасеты на левую и правую подвыборку 
        по признаку feature на основе порога threshold
        """
        left_indexes = np.where(X[:,feature] <= threshold)[0]
        right_indexes = np.where(X[:,feature] > threshold)[0]
        return X[left_indexes], Y[left_indexes], X[right_indexes], Y[right_indexes]

    def _find_best_split(self, X: np.ndarray, Y: np.ndarray):
        """
        Находит лучшее разделение данных на левую и правую подвыборку
        """
        best_feature, best_threshold, best_score = None, None, np.inf

        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:,feature])
            for threshold in thresholds:
                x_left, y_left, x_right, y_right = self._split_dataset(X,Y,feature,threshold)

                if self.classification:
                    score = (len(y_left) * self._entropy(y_left) + \
                             len(y_right) * self._entropy(y_right)) / len(Y)
                else:
                    score = (len(y_left) * self._MSE(y_left) + \
                             len(y_right) * self._MSE(y_right)) / len(Y)
                if score < best_score:
                    best_feature, best_threshold, best_score = feature, threshold, score
        return best_feature, best_threshold

    def _build_tree(self, X: np.ndarray, Y: np.ndarray, depth=0) -> Node:
        if depth == self.max_depth or len(X) < self.min_samples_split:
            if self.classification:
                return self.Node(value=Counter(Y).most_common(1)[0][0])
            else:
                return self.Node(value=np.mean(Y))
            
        feature, threshold = self._find_best_split(X,Y)
        x_left, y_left, x_right, y_right = self._split_dataset(X,Y,feature,threshold)
        left_child = self._build_tree(x_left,y_left,depth=depth + 1)
        right_child = self._build_tree(x_right,y_right,depth=depth + 1)

        return self.Node(feature=feature,threshold=threshold,left=left_child,right=right_child)

    def fit(self, X: np.ndarray, Y: np.ndarray):
        self.tree_ = self._build_tree(X,Y)
        return self

    def _predict_single(self, x: np.ndarray, node: Node) -> int | float:
        if node.feature is None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self._predict_single(x, node.left)
        else:
            return self._predict_single(x, node.right)

    def predict(self, X: np.ndarray) -> list[float] | list[int]:
        y_pred = np.zeros(len(X), dtype=int if self.classification else float)
        for i in range(X.shape[0]):
            y_pred[i] = self._predict_single(X[i], self.tree_)
        return y_pred

In [102]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

x,y = make_classification(
    n_samples=2000,
    n_clusters_per_class=1,
    n_features=4,
    n_classes=3
)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [103]:
from sklearn.metrics import classification_report

crt = CART(classification=True)
crt.fit(x_train,y_train)
predict = crt.predict(x_test)

print(classification_report(y_test,predict))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95       129
           1       0.96      0.95      0.96       137
           2       0.99      0.99      0.99       134

    accuracy                           0.96       400
   macro avg       0.96      0.96      0.96       400
weighted avg       0.96      0.96      0.96       400



In [104]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)

pred = dt.predict(x_test)
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95       129
           1       0.98      0.93      0.95       137
           2       1.00      0.99      0.99       134

    accuracy                           0.96       400
   macro avg       0.97      0.97      0.97       400
weighted avg       0.97      0.96      0.97       400



In [105]:
from sklearn.datasets import make_regression
x,y = make_regression(
    n_samples=2000,
    n_features=4
)

x_train, x_test,  y_train,  y_test = train_test_split(x,y, test_size=0.2,random_state = 4)

In [119]:
crt = CART()
crt.fit(x_train,y_train)
predict = crt.predict(x_test)
regression_metrics(y_test,predict)

C:\Users\Sviat\Desktop\mashobych\venv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Sviat\Desktop\mashobych\venv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


MAE: 15.280133349409306
MSE: 384.859274878438
RMSE: 19.617830534451
MAPE: 0.9248141497033697
R^2: 0.9462971646342391


In [120]:
dt = DecisionTreeRegressor(random_state = 0)
dt.fit(x_train,y_train)

predict = dt.predict(x_test)
regression_metrics(y_test,predict)

MAE: 14.601753557748339
MSE: 400.112947822617
RMSE: 20.002823496262145
MAPE: 0.8912937679269818
R^2: 0.9441686840692244
